In [167]:
import pandas as pd
import numpy as np

In [168]:
df = pd.read_excel (r'C:\Users\jeanp\Desktop\IMDB-Movie-Reviews-Large-Dataset-50k-master/train.xlsx')
df.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


TFIDF

In [169]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

In [170]:
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

nltk.download('stopwords')
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [171]:
tfidf = TfidfVectorizer(analyzer=preprocess)
X = df['Reviews']
y = df['Sentiment']
X= tfidf.fit_transform(X)

In [172]:
X

<25000x37 sparse matrix of type '<class 'numpy.float64'>'
	with 675959 stored elements in Compressed Sparse Row format>

In [173]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [174]:
clf=LinearSVC()
clf.fit(X_train,y_train)

LinearSVC()

In [175]:
y_pred = clf.predict(X_test)

In [176]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         neg       0.62      0.60      0.61      2480
         pos       0.62      0.63      0.63      2520

    accuracy                           0.62      5000
   macro avg       0.62      0.62      0.62      5000
weighted avg       0.62      0.62      0.62      5000



In [194]:
x= 'bad'
y= 'good'
x=preprocess(x)
y=preprocess(y)
vec = tfidf.transform([x])
vec1 = tfidf.transform([y])

In [196]:
clf.predict(vec)

array(['neg'], dtype=object)

In [197]:
clf.predict(vec1)

array(['pos'], dtype=object)

In [180]:
import pickle

In [198]:
pickle.dump(clf,open('custom_model','wb'))

In [199]:
dff = pd.read_csv('moviereviews.tsv', sep ='\t')
dff.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [200]:
tfidf = TfidfVectorizer(analyzer=preprocess)
XX = dff['review'].values.astype('U')
yy = dff['label']
XX= tfidf.fit_transform(XX)

In [201]:
XX_train, XX_test, yy_train, yy_test = train_test_split(XX,yy,test_size=0.2, random_state=0)

In [203]:
loaded_model = pickle.load(open('custom_model', 'rb'))
result = loaded_model.score(XX_test, yy_test)
print(result)

0.605
